# Sales in Hive

Siny P Raphel

## Load Data in hive table

```create table sales (unnamed int, order_id int, product varchar(250), quantity int, price_each float,order_date date, purchase_address varchar(500), month int, city varchar(100))
row format delimited
fields terminated by ','
lines terminated by '\n'
stored as textfile
tblproperties ("skip.header.line.count"="1"); ```

```load data local inpath '/root/sales-data.csv' into table sales; ```

#load data from root to sales table



There was no way to create a column and fill values to it. Therefore created a new tble with the select statement as shown below. New columns sales and hour is added. There was error in converting order_date to datetime format. So changed data type using multiple functions.

```create table sales1 as 
select order_id, product, quantity, price_each, 
from_unixtime(unix_timestamp(order_date, 'mm/dd/yy HH:MM'), 'mm/dd/yy HH:MM') as order_date, 
purchase_address, month, city, 
price_each * quantity as sales,
from_unixtime(unix_timestamp(order_date, 'mm/dd/yy HH:MM'), 'H') as hour
from sales_data;```

###  What was the best month for sales? How much was earned that month?

```select month, sum(sales) as total_sales
from sales1
group by month
order by total_sales desc
limit 1;```

### What city sold the most product?

```select city, sum(quantity) as total
from sales1
group by city
order by total desc
limit 1;```

### What time should we display advertisemens to maximize the likelihood of customer’s buying product?

```select hour, sum(quantity) as total
from sales1
group by hour
order by total desc
limit 1;```

### What products are most often sold together?

The following code concats all the products within each order_id and returns order_id, concatenated product and count

```SELECT order_id, CONCAT_WS(',',collect_set(product)) AS prd, count(order_id) as ct
FROM sales1
GROUP BY order_id
having ct >1;```

The above query is used as a subquery 

```select prd, count(prd) as prd_ct
from (SELECT order_id, CONCAT_WS(',',collect_set(product)) AS prd, count(order_id) as ct
FROM sales1
GROUP BY order_id
having ct >1) as orders
group by prd
order by prd_ct desc
limit 1;```

### What product sold the most? Why do you think it sold the most?

```select product, sum(quantity) as total
from sales1
group by product
order by total desc
limit 1;```